## Resampling Monthly Data 

In [1]:
import pandas as pd
from IPython.display import clear_output

In [2]:
weather = pd.read_csv('./data/weather_data.csv')
weather['date'] = pd.to_datetime(weather.date)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8700 entries, 0 to 8699
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          8700 non-null   datetime64[ns]
 1   county        8700 non-null   object        
 2   maxtempF      8700 non-null   float64       
 3   mintempF      8700 non-null   float64       
 4   avgtempF      8700 non-null   float64       
 5   totalSnow_cm  8700 non-null   float64       
 6   humid         8700 non-null   float64       
 7   wind          8700 non-null   float64       
 8   precip        8700 non-null   float64       
 9   sunHour       8700 non-null   float64       
 10  lat           8700 non-null   float64       
 11  long          8700 non-null   float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 815.8+ KB


At this point the weather data is aggregated on a monthly basis.  Since we are predicting the effect that weather might have on forest fires it may be beneficial to look at what the weather has been like in a given county for the months leading up to a potential fire.  If it has not rained in months, for example, this might be a valuable predictive feature.  Therefore we resample some selected features on a 3 month basis and add these new features to our data frame.

In [3]:
county_list = list(weather['county'].unique())

county_dfs = []
for county in county_list:
    clear_output()
    print(county)
    x = weather[weather['county'] == county]
    x.set_index('date', inplace=True)
    county_dfs.append(x)

iters = 0
for df in county_dfs:
    county = df['county'][0]
    trans_df = df[['precip']].rolling(3).sum()
    temp_df = df[['avgtempF', 'humid']].rolling(3).mean()
    temp_df['county'] = county
    temp_df['precip'] = trans_df['precip']
    temp_df.reset_index(inplace = True)
    if iters == 0:
        out = temp_df.copy()
    else:
        out = pd.concat([out, temp_df], ignore_index = True)
    iters += 1
    clear_output()
    print(iters)
    


out['date'] = out['date'].astype(str)
out['date'] = out['date'].map(lambda s: s[0:-3])

weather['date'] = weather['date'].astype(str)
weather['date'] = weather['date'].map(lambda s: s[0:-3])

weather['q_avgtempF'] = out['avgtempF']
weather['q_avghumid'] = out['humid']
weather['q_sumprecip'] = out['precip']


weather.info()

58
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8700 entries, 0 to 8699
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          8700 non-null   object 
 1   county        8700 non-null   object 
 2   maxtempF      8700 non-null   float64
 3   mintempF      8700 non-null   float64
 4   avgtempF      8700 non-null   float64
 5   totalSnow_cm  8700 non-null   float64
 6   humid         8700 non-null   float64
 7   wind          8700 non-null   float64
 8   precip        8700 non-null   float64
 9   sunHour       8700 non-null   float64
 10  lat           8700 non-null   float64
 11  long          8700 non-null   float64
 12  q_avgtempF    8584 non-null   float64
 13  q_avghumid    8584 non-null   float64
 14  q_sumprecip   8584 non-null   float64
dtypes: float64(13), object(2)
memory usage: 1019.7+ KB


Next we need to merge the fire data to the completed weather data.  Remember from before that the ```UNIT_ID``` feature from the fire data contains some cases where one unit id corresponded to two counties.  To correect for this we assign those fires to only one of the counties so that we can complete the merge.

In [4]:
fire = pd.read_csv('./data/fire_data.csv')

def remove_dash(x):
    if '-' in x:
        county = x.split('- ')
        return county[1]
    else:
        return x
        
fire['UNIT_ID'] = fire['UNIT_ID'].apply(remove_dash)
final = pd.merge(weather, fire, how = 'left', left_on = ['county', 'date'], right_on = ['UNIT_ID', 'ALARM_DATE'])
final.head()

,date,county,maxtempF,mintempF,avgtempF,totalSnow_cm,humid,wind,precip,sunHour,...,q_sumprecip,UNIT_ID,FIRE_NAME,ALARM_DATE,CONT_DATE,CAUSE,REPORT_AC,GIS_ACRES,SHAPE_Length,SHAPE_Area
0,2008-07,Sierra County,86.290323,46.645161,76.709677,0.0,32.709677,5.451613,0.0,14.396774,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-07,Sacramento County,97.290323,63.419355,86.516129,0.0,39.838710,4.741935,0.0,13.741935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-07,Santa Barbara County,89.129032,59.709677,80.548387,0.0,41.451613,7.354839,0.0,13.164516,...,0.010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-07,Calaveras County,96.419355,51.290323,87.032258,0.0,33.580645,5.387097,0.0,14.022581,...,0.068065,Calaveras County,SERPENTINE,2008-07,2008/07/29 00:00:00+00,1.0,177.0,176.763400,7368.548447,1.146732e+06
4,2008-07,Calaveras County,96.419355,51.290323,87.032258,0.0,33.580645,5.387097,0.0,14.022581,...,0.068065,Calaveras County,POOL 2 - MAIN,2008-07,2008/07/29 00:00:00+00,10.0,73.5,73.444778,3200.154767,4.800265e+05


Next we drop columns that will not be used for modeling.

In [5]:
final.drop(columns = ['UNIT_ID', 'ALARM_DATE', 'CONT_DATE', 'SHAPE_Length', 'SHAPE_Area', 'REPORT_AC'], inplace = True)

final.head()

,date,county,maxtempF,mintempF,avgtempF,totalSnow_cm,humid,wind,precip,sunHour,lat,long,q_avgtempF,q_avghumid,q_sumprecip,FIRE_NAME,CAUSE,GIS_ACRES
0,2008-07,Sierra County,86.290323,46.645161,76.709677,0.0,32.709677,5.451613,0.0,14.396774,39.58,-120.52,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-07,Sacramento County,97.290323,63.419355,86.516129,0.0,39.838710,4.741935,0.0,13.741935,38.45,-121.34,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-07,Santa Barbara County,89.129032,59.709677,80.548387,0.0,41.451613,7.354839,0.0,13.164516,34.54,-120.04,72.937634,34.079570,0.010000,NaN,NaN,NaN
3,2008-07,Calaveras County,96.419355,51.290323,87.032258,0.0,33.580645,5.387097,0.0,14.022581,38.18,-120.56,65.217204,38.434409,0.068065,SERPENTINE,1.0,176.763400
4,2008-07,Calaveras County,96.419355,51.290323,87.032258,0.0,33.580645,5.387097,0.0,14.022581,38.18,-120.56,65.217204,38.434409,0.068065,POOL 2 - MAIN,10.0,73.444778


In [6]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11149 entries, 0 to 11148
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          11149 non-null  object 
 1   county        11149 non-null  object 
 2   maxtempF      11149 non-null  float64
 3   mintempF      11149 non-null  float64
 4   avgtempF      11149 non-null  float64
 5   totalSnow_cm  11149 non-null  float64
 6   humid         11149 non-null  float64
 7   wind          11149 non-null  float64
 8   precip        11149 non-null  float64
 9   sunHour       11149 non-null  float64
 10  lat           11149 non-null  float64
 11  long          11149 non-null  float64
 12  q_avgtempF    10988 non-null  float64
 13  q_avghumid    10988 non-null  float64
 14  q_sumprecip   10988 non-null  float64
 15  FIRE_NAME     4027 non-null   object 
 16  CAUSE         4015 non-null   float64
 17  GIS_ACRES     4040 non-null   float64
dtypes: float64(15), object(3)


There are some missing values that need to be imputed.  Some of these, such as where the ```GIS_ACRES``` is null we can just change that to 0 since the null represents that there was no fire.  For the missing causes we will assign a 0 if there was no fire, but will assign the most common cause for the instances where we do have a fire, but a null in the cause.

In [7]:
final['FIRE_NAME'].fillna('no_fire', inplace = True)

final['GIS_ACRES'].fillna(0, inplace = True)

mode_cause = final['CAUSE'].value_counts().index[0]

x , y = final.shape
for i in range(x):
    clear_output()
    print(f"{i} of {x-1}")
    if final.iloc[i,-1] != 0 and pd.isnull(final.iloc[i,-2]):
        final.iloc[i,-2] = mode_cause

final['CAUSE'].fillna(0, inplace = True)

11148 of 11148


Since we used a 3 month average for some of the resampled fields there are two records for each county that contain null weather data corresponding to the first two months in our time interval.  These can be dropped.

In [8]:
final.dropna(inplace = True)

final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10988 entries, 2 to 11148
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          10988 non-null  object 
 1   county        10988 non-null  object 
 2   maxtempF      10988 non-null  float64
 3   mintempF      10988 non-null  float64
 4   avgtempF      10988 non-null  float64
 5   totalSnow_cm  10988 non-null  float64
 6   humid         10988 non-null  float64
 7   wind          10988 non-null  float64
 8   precip        10988 non-null  float64
 9   sunHour       10988 non-null  float64
 10  lat           10988 non-null  float64
 11  long          10988 non-null  float64
 12  q_avgtempF    10988 non-null  float64
 13  q_avghumid    10988 non-null  float64
 14  q_sumprecip   10988 non-null  float64
 15  FIRE_NAME     10988 non-null  object 
 16  CAUSE         10988 non-null  float64
 17  GIS_ACRES     10988 non-null  float64
dtypes: float64(15), object(3)


All of our data was provided in imperial units with the exception of snow, which was given in centimeters.  These units are converted next.

In [9]:
final['totalSnow'] = final['totalSnow_cm'] / 2.54
final.drop(columns = ['totalSnow_cm'], inplace = True)

Finally, the columns of our data frame are reordered to collect all of the weather data in contiguous columns simply for ease of readability.  The dataset is now complete and saved as ```combined.csv``` for use in eda and modeling.

In [10]:
final = final[['date', 'county', 'maxtempF', 'mintempF', 'avgtempF', 'totalSnow', 'humid', 'wind', 'precip', 
               'q_avgtempF', 'q_avghumid', 'q_sumprecip', 'sunHour', 'FIRE_NAME', 'CAUSE', 'lat', 'long', 'GIS_ACRES']]

final.to_csv('./data/combined.csv', index = False)